In [ ]:

!pip install nltk


In [ ]:
#  1. Imports
import nltk
from collections import defaultdict, Counter
import math, random

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#  2. Load and preprocess dataset

file_path = "Guj_3000.txt"

with open(file_path, "r", encoding="utf-8") as f:
    data = f.read().splitlines()

# Tokenize sentences (Gujarati text is tokenized by spaces)
# Tokenize by splitting on spaces (works for Gujarati text)
sentences = [sent.split() for sent in data]


# Add start <s> and end </s> tokens for quadrigrams
processed = []
for sent in sentences:
    processed.append(["<s>", "<s>", "<s>"] + sent + ["</s>"])

print("Sample sentence tokens:", processed[0])


Sample sentence tokens: ['<s>', '<s>', '<s>', '1.', 'ગાંધીનગરમાં', 'હાલમાં', 'રાજકારણ', 'ક્ષેત્રે', 'શાળા', 'વિકાસ', 'સમિતિએ', 'સ્થગિત', 'થયું;', 'તે', 'ઉપરાંત', '88', 'મુદ્દાઓ', 'પર', 'ભાર', 'મૂકાયો', 'અને', 'કારોબારમાં', 'વૃદ્ધિ', 'નોંધાઈ.', 'લક્ષ્યાંકો', 'આગામી', 'છ', 'મહિનામાં', 'પૂર્ણ', 'કરવાનો', 'ઉદ્દેશ', 'છે.', '</s>']


In [ ]:
#  3. Build N-gram counts (up to 4-gram)
from collections import Counter

unigrams = Counter()
bigrams = Counter()
trigrams = Counter()
quadrigrams = Counter()

for sent in processed:
    for i in range(len(sent)):
        unigrams[tuple(sent[i:i+1])] += 1
        if i < len(sent)-1:
            bigrams[tuple(sent[i:i+2])] += 1
        if i < len(sent)-2:
            trigrams[tuple(sent[i:i+3])] += 1
        if i < len(sent)-3:
            quadrigrams[tuple(sent[i:i+4])] += 1

print("Unigrams:", len(unigrams))
print("Quadrigrams:", len(quadrigrams))



Unigrams: 1370
Quadrigrams: 10540


In [ ]:
#  4. Katz Backoff Implementation

# For unseen higher n-grams, fall back to lower n-gram probability.

def katz_backoff_prob(ngram, d=0.5):
    """
    Compute Katz backoff probability for a given 4-gram.
    """
    if quadrigrams[ngram] > 0:
        return (quadrigrams[ngram] - d) / trigrams[ngram[:-1]]
    elif trigrams[ngram[1:]] > 0:
        return (trigrams[ngram[1:]] - d) / bigrams[ngram[1:-1]]
    elif bigrams[ngram[2:]] > 0:
        return (bigrams[ngram[2:]] - d) / unigrams[ngram[2:-1]]
    else:
        return unigrams[ngram[3:]] / sum(unigrams.values())


In [ ]:
#  5. Kneser–Ney Smoothing Implementation

# Discounted continuation probability for unseen n-grams

def kneser_ney_prob(ngram, d=0.75):
    quad_count = quadrigrams[ngram]
    tri_count = trigrams[ngram[:-1]]

    if tri_count > 0:
        prob = max(quad_count - d, 0) / tri_count
        # continuation probability
        continuation = len([1 for q in quadrigrams if q[1:] == ngram[1:]]) / len(quadrigrams)
        prob += (d / tri_count) * continuation
    else:
        prob = unigrams[ngram[3:]] / sum(unigrams.values())

    return prob


In [ ]:
# 📌 6. Sentence Generation (Greedy + Beam Search)
VOCAB = list(set([w for s in processed for w in s]))

def generate_sentence(model="katz", max_len=20, method="greedy", beam_size=20):
    sent = ["<s>", "<s>", "<s>"]

    for _ in range(max_len):
        candidates = {}
        for w in VOCAB:
            ngram = tuple(sent[-3:] + [w])
            if model == "katz":
                prob = katz_backoff_prob(ngram)
            else:
                prob = kneser_ney_prob(ngram)
            candidates[w] = prob

        if method == "greedy":
            next_word = max(candidates, key=candidates.get)
            sent.append(next_word)
            if next_word == "</s>":
                break

        elif method == "beam":
            # Beam Search
            beams = [[sent, 0.0]]
            for _ in range(max_len):
                new_beams = []
                for seq, score in beams:
                    for w in VOCAB:
                        ngram = tuple(seq[-3:] + [w])
                        if model == "katz":
                            prob = katz_backoff_prob(ngram)
                        else:
                            prob = kneser_ney_prob(ngram)
                        new_beams.append([seq+[w], score+math.log(prob+1e-10)])
                # keep top k beams
                beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
                if beams[0][0][-1] == "</s>":
                    break
            sent = beams[0][0]
            break
    return " ".join(sent[3:])


In [14]:
def generate_sentence(model="katz", max_len=20, method="greedy", beam_size=20, top_k=50):
    sent = ["<s>", "<s>", "<s>"]

    for _ in range(max_len):
        candidates = {}
        for w in VOCAB:
            if w == "<s>":
                continue
            ngram = tuple(sent[-3:] + [w])
            if model == "katz":
                prob = katz_backoff_prob(ngram)
            else:
                prob = kneser_ney_prob(ngram)
            candidates[w] = prob

        # keep only top_k candidates
        candidates = dict(sorted(candidates.items(), key=lambda x: x[1], reverse=True)[:top_k])

        if method == "greedy":
            next_word = max(candidates, key=candidates.get)
            sent.append(next_word)
            if next_word == "</s>":
                break

        elif method == "beam":
            beams = [[sent, 0.0]]
            for _ in range(max_len):
                new_beams = []
                for seq, score in beams:
                    for w, prob in candidates.items():  # 🚀 now only top_k words
                        new_beams.append([seq+[w], score+math.log(prob+1e-10)])
                beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
                if beams[0][0][-1] == "</s>":
                    break
            sent = beams[0][0]
            break
    return " ".join(sent[3:])


In [15]:
print("🔹 Katz Backoff - Greedy")
for i in range(5):
    print(generate_sentence(model="katz", method="greedy"))

print("\n🔹 Katz Backoff - Beam Search")
for i in range(5):
    print(generate_sentence(model="katz", method="beam"))

print("\n🔹 Kneser–Ney - Greedy")
for i in range(5):
    print(generate_sentence(model="kn", method="greedy"))

print("\n🔹 Kneser–Ney - Beam Search")
for i in range(5):
    print(generate_sentence(model="kn", method="beam"))


🔹 Katz Backoff - Greedy
પર ભાર મૂકાયો અને સ્થાનિક સ્તરે સહકાર વધ્યો. ઓનલાઇન પોર્ટલ પણ શરૂ કરવામાં આવ્યું છે. </s>
પર ભાર મૂકાયો અને સ્થાનિક સ્તરે સહકાર વધ્યો. ઓનલાઇન પોર્ટલ પણ શરૂ કરવામાં આવ્યું છે. </s>
પર ભાર મૂકાયો અને સ્થાનિક સ્તરે સહકાર વધ્યો. ઓનલાઇન પોર્ટલ પણ શરૂ કરવામાં આવ્યું છે. </s>
પર ભાર મૂકાયો અને સ્થાનિક સ્તરે સહકાર વધ્યો. ઓનલાઇન પોર્ટલ પણ શરૂ કરવામાં આવ્યું છે. </s>
પર ભાર મૂકાયો અને સ્થાનિક સ્તરે સહકાર વધ્યો. ઓનલાઇન પોર્ટલ પણ શરૂ કરવામાં આવ્યું છે. </s>

🔹 Katz Backoff - Beam Search
પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર
પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર
પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર
પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર
પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર પર

🔹 Kneser–Ney - Greedy
601. બનાસકાંઠામાં આ મહિને પર્યાવરણ ક્ષેત્રે ટ્રાફિક વિભાગએ ખુલાસો કર્યો; પછી 39 મુદ્દાઓ પર ભાર મૂકાયો અને સ્થાનિક સ્તરે સહકાર
601. બનાસકાંઠામાં આ મહિને પર્યાવરણ ક્ષેત્રે ટ્ર